# importing libraries

In [1]:
# import libraries
import pandas as pd
import numpy as np
import os

# importing file

In [2]:
# importing library
path = r"C:\your\base\directory"
input_dir = os.path.join(path, '02 Data', 'Prepared Data')
pickle_file = os.path.join(input_dir, 'ords_prods_merge_derived.pkl')

In [3]:
# Load the pickle file into a DataFrame
ords_prods_merge = pd.read_pickle(pickle_file)

In [5]:
ords_prods_merge.shape

(32404859, 18)

# Aggregating mean for entire data

In [6]:
ords_prods_merge.groupby('department_id')['order_number'].mean()

department_id
1     15.457838
2     17.277920
3     17.170395
4     17.811403
5     15.215751
6     16.439806
7     17.225802
8     15.340650
9     15.895474
10    20.197148
11    16.170638
12    15.887671
13    16.583536
14    16.773669
15    16.165037
16    17.665606
17    15.694469
18    19.310397
19    17.177343
20    16.473447
21    22.902379
Name: order_number, dtype: float64

# How do the results for the entire dataframe differ from those of the subset?

The values in the subset are slightly higher than those in the entire dataset for most department_id groups. For example, for department_id 1, the subset has a mean of 15.63, while the entire dataset has 15.46. Similar differences can be observed for other departments. 
For department_id 21, the values in the subset (23.26) are slightly higher compared to the entire dataset (22.90). This may indicate a small bias in the subset for the higher order_number values.

# Loyalty flag

In [7]:
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

C:\Users\User\AppData\Local\Temp\ipykernel_8064\1681540670.py:1: FutureWarning: The provided callable <function max at 0x000001A31164F9C0> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)


In [8]:
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform("max")

In [9]:
ords_prods_merge.head(15)

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,product_name,aisle_id,department_id,prices,price_range_loc,busiest day,busiest_days,busiest_period_of_day,max_order
0,2,202279,3,5,9,8.0,33120.0,1.0,1.0,both,Organic Egg Whites,86,16,11.3,Mid-range product,Regularly busy,Regular days,Most orders,8
1,2,202279,3,5,9,8.0,28985.0,2.0,1.0,both,Michigan Organic Kale,83,4,13.4,Mid-range product,Regularly busy,Regular days,Most orders,8
2,2,202279,3,5,9,8.0,9327.0,3.0,0.0,both,Garlic Powder,104,13,3.6,Low-range product,Regularly busy,Regular days,Most orders,8
3,2,202279,3,5,9,8.0,45918.0,4.0,1.0,both,Coconut Butter,19,13,8.4,Mid-range product,Regularly busy,Regular days,Most orders,8
4,2,202279,3,5,9,8.0,30035.0,5.0,0.0,both,Natural Sweetener,17,13,13.7,Mid-range product,Regularly busy,Regular days,Most orders,8
5,2,202279,3,5,9,8.0,17794.0,6.0,1.0,both,Carrots,83,4,10.7,Mid-range product,Regularly busy,Regular days,Most orders,8
6,2,202279,3,5,9,8.0,40141.0,7.0,1.0,both,Original Unflavored Gelatine Mix,105,13,11.5,Mid-range product,Regularly busy,Regular days,Most orders,8
7,2,202279,3,5,9,8.0,1819.0,8.0,1.0,both,All Natural No Stir Creamy Almond Butter,88,13,11.5,Mid-range product,Regularly busy,Regular days,Most orders,8
8,2,202279,3,5,9,8.0,43668.0,9.0,0.0,both,Classic Blend Cole Slaw,123,4,7.5,Mid-range product,Regularly busy,Regular days,Most orders,8
9,3,205970,16,5,17,12.0,33754.0,1.0,1.0,both,Total 2% with Strawberry Lowfat Greek Strained...,120,16,11.8,Mid-range product,Regularly busy,Regular days,Average orders,25


In [10]:
# Creating a 'Order Frequency' Label Flag using loc Function
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [11]:
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [12]:
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [13]:
ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

loyalty_flag
Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: count, dtype: int64

# Create a spending flag for each user based on the average price across all their orders

In [21]:
# Calculate the mean price for each user
ords_prods_merge['avg_product_price'] = ords_prods_merge.groupby(['user_id'])['prices'].transform('mean')

In [23]:
ords_prods_merge.loc[ords_prods_merge['avg_product_price'] >= 10, 'spending_flag'] = 'High spender'
ords_prods_merge.loc[ords_prods_merge['avg_product_price'] < 10, 'spending_flag'] = 'Low spender'

In [25]:
ords_prods_merge['spending_flag'].value_counts(dropna = False)

spending_flag
Low spender     31770614
High spender      634245
Name: count, dtype: int64

In [26]:
# Checking Releavent Columns
ords_prods_merge[['user_id', 'avg_product_price', 'spending_flag']].head(15)

,user_id,avg_product_price,spending_flag
0,202279,8.618889,Low spender
1,202279,8.618889,Low spender
2,202279,8.618889,Low spender
3,202279,8.618889,Low spender
4,202279,8.618889,Low spender
5,202279,8.618889,Low spender
6,202279,8.618889,Low spender
7,202279,8.618889,Low spender
8,202279,8.618889,Low spender
9,205970,8.681734,Low spender


# Create an order frequency flag that marks the regularity of a user’s ordering behavior according to the median in the “days_since_prior_order” column

In [27]:
# Aggregating Data with transform() Function
ords_prods_merge['median_days_between_orders'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform('median')

In [29]:
# Checking Relevant Columns 
ords_prods_merge[['user_id', 'days_since_prior_order', 'median_days_between_orders']].head(15)

,user_id,days_since_prior_order,median_days_between_orders
0,202279,8.0,24.0
1,202279,8.0,24.0
2,202279,8.0,24.0
3,202279,8.0,24.0
4,202279,8.0,24.0
5,202279,8.0,24.0
6,202279,8.0,24.0
7,202279,8.0,24.0
8,202279,8.0,24.0
9,205970,12.0,8.0


In [31]:
# Creating a 'Order Frequency' Label Flag using loc Function
ords_prods_merge.loc[ords_prods_merge['median_days_between_orders'] > 20, 'order_frequency_flag'] = 'Non-frequent customer'
ords_prods_merge.loc[(ords_prods_merge['median_days_between_orders'] <= 20) & (ords_prods_merge['median_days_between_orders'] > 10), 'order_frequency_flag'] = 'Regular customer'
ords_prods_merge.loc[ords_prods_merge['median_days_between_orders'] <= 10, 'order_frequency_flag'] = 'Frequent customer'

In [33]:
# Examining Value Counts on 'order_frequency_flag'
ords_prods_merge['order_frequency_flag'].value_counts(dropna = False)

order_frequency_flag
Frequent customer        21559853
Regular customer          7208564
Non-frequent customer     3636437
NaN                             5
Name: count, dtype: int64

In [34]:
# Checking Relevant Columns of Updated Dataframe
ords_prods_merge[['user_id', 'days_since_prior_order', 'median_days_between_orders', 'order_frequency_flag']].head(15)

,user_id,days_since_prior_order,median_days_between_orders,order_frequency_flag
0,202279,8.0,24.0,Non-frequent customer
1,202279,8.0,24.0,Non-frequent customer
2,202279,8.0,24.0,Non-frequent customer
3,202279,8.0,24.0,Non-frequent customer
4,202279,8.0,24.0,Non-frequent customer
5,202279,8.0,24.0,Non-frequent customer
6,202279,8.0,24.0,Non-frequent customer
7,202279,8.0,24.0,Non-frequent customer
8,202279,8.0,24.0,Non-frequent customer
9,205970,12.0,8.0,Frequent customer


# Exporting data

In [35]:
ords_prods_merge.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_group.pkl'))